<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd

# Вместо psycopg2 используем библиотеку sqlalchemy чтобы не воевать с предупреждениями.
from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
import re
import numpy as np

# Импортируем класс con, с параметрами подключения.
from servise.connect import con

In [2]:
# Создаем экземпляр класса con для вызова функций, возращаюших параметры подключения из юнита 1.
c = con 

# Создаем подключение. Для воспроизведения подключения нужно написать класс с одноименными функциями, 
# возращающими требуемые параметры подключения, либо заменить значения в фигурных скобках на параметры подключения из юнита 1.
connection = create_engine(f'postgresql+psycopg2://{c.get_user()}:{c.get_pas()}@{c.get_host()}:{c.get_port()}/{c.get_name()}')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
query_3_1 = f'''SELECT 
                    COUNT(id)
                FROM public.vacancies
            '''

In [4]:
count_vacancies = pd.read_sql_query(query_3_1, connection)
print(f'Количество вакансий в базе данных: {count_vacancies.values[0][0]}')

Количество вакансий в базе данных: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
query_3_2 = f'''SELECT 
                    COUNT(id)
                FROM public.employers
            '''

In [6]:
count_employers = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей в базе данных: {count_employers.values[0][0]}')

Количество работодателей в базе данных: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
query_3_3 = f'''SELECT 
                    COUNT(id)
                FROM public.areas
            '''

In [8]:
count_areas = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов в базе данных: {count_areas.values[0][0]}')

Количество регионов в базе данных: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [9]:
query_3_4 = f'''SELECT
                    COUNT(id)
                FROM public.industries
            '''

In [10]:
count_industries = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности в базе данных: {count_industries.values[0][0]}')

Количество сфер деятельности в базе данных: 294


### Выводы по предварительному анализу данных 
***
* База данных содержит сведенья о 49197 вакансиях от 23501 работодателя из 1362 регионов в 294 различных сферах деятельности.
* По состоянию на 1 октября 2021 года в России насчитывается 1118 городов, следовательно можно предпложить что в базе данных имеются вакансии за пределами России.
* В среднем на каждого работодателя приходится 2,09 вакансии. Скорее всего в базе представлены как крупные фирмы, предлагающие несколько вакансий, так и мелкие компании с одним предложением.
* Количество различных сфер деятельности значительно меньше количества работодателей. В среднем на каждую сферу деятельности приходится около 80 работодателей. Вероятнее всего распределение работодателей по сферам деятельности неравномерное - существуют как популярные сферы деятельности так и достаточно редкие.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [11]:
query_4_1 = f'''SELECT 
                    a.name AS area,
                    COUNT(v.id) AS cnt
                FROM public.vacancies AS v
                JOIN public.areas AS a 
                    ON v.area_id = a.id
                GROUP BY a.name
                ORDER BY cnt DESC
            '''

In [12]:
cnt_vac_in_area = pd.read_sql_query(query_4_1, connection)
print(f"Список регионов лидеров по количеству вакансий: {list(cnt_vac_in_area.loc[0:4,'area'])}")

# Выведем первые 5 строк датафрейма для оценки количества вакансий в регионах лидерах.
display(cnt_vac_in_area.head(5))

Список регионов лидеров по количеству вакансий: ['Москва', 'Санкт-Петербург', 'Минск', 'Новосибирск', 'Алматы']


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [13]:
query_4_2 = f'''SELECT 
                    COUNT(id)
                FROM public.vacancies
                WHERE salary_from IS NOT NULL
                    OR salary_to IS NOT NULL                
            '''

In [36]:
count_vac_have_salary = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий с хотя бы одним заполненым полем из двух полей '
      f'с зарпалтой: {count_vac_have_salary.values[0][0]}')

Количество вакансий с хотя бы одним заполненым полем из двух полей с зарпалтой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
query_4_3 = f'''SELECT 
                    ROUND(AVG(salary_from), 0) AS from,
                    ROUND(AVG(salary_to), 0) AS to
                FROM public.vacancies
            '''

In [20]:
avg_salary = pd.read_sql_query(query_4_3, connection)
print(f'Среднее значение зарплаты составляет:'
      f'\n - для нижней границы - {avg_salary.iloc[0][0]} рублей'
      f'\n - для верхней границы - {avg_salary.iloc[0][1]} рублей')


Среднее значение зарплаты составляет:
 - для нижней границы - 71065.0 рублей
 - для верхней границы - 110537.0 рублей


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [21]:
query_4_4 = f'''SELECT 
                    schedule,
                    employment,
                    COUNT(id) AS count_vac
                FROM public.vacancies
                GROUP BY schedule, employment
                ORDER BY 3 DESC
            '''

In [22]:
sched_empl_count_vac = pd.read_sql_query(query_4_4, connection)
print(f'Второе по популярности сочетание типа рабочего графика и '
      f'типа трудоустройства: {sched_empl_count_vac.values[1][0:2]}')

Второе по популярности сочетание типа рабочего графика и типа трудоустройства: ['Удаленная работа' 'Полная занятость']


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
query_4_5 = f'''SELECT 
                    experience,
                    COUNT(id) AS count_vac
                FROM public.vacancies
                GROUP BY experience
                ORDER BY count_vac
            '''

In [38]:

exp_count_vac = pd.read_sql_query(query_4_5, connection)
print(f'Требуемый опыт работы в порядке возрастания по количеству вакансий, '
      f'в которых указан данный вариант опыта: \n{list(exp_count_vac.loc[:, "experience"])}')

Требуемый опыт работы в порядке возрастания по количеству вакансий, в которых указан данный вариант опыта: 
['Более 6 лет', 'Нет опыта', 'От 3 до 6 лет', 'От 1 года до 3 лет']


### Выводы по детальному анализу вакансий
***
* Около трети вакансий (28,85%) предлагается в пяти регионах: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Причем на Москву приходится 10,84% всех представленных вакансий. Список регионов подтверждает предыдущий вывод о том, что база охватывает не только регионы России. Отношение количества предложенных в регионах вакансий к населению этих регионов показывает иное распределение регионов: Новосибирск, Минск, Алматы, Санкт-Петербург, Москва. Можно сделать вывод, что база содержит достаточно подробную выборку вакансий по различным регионам, а не центрированна на каком-то одном. 
* Менее половины (48,93%) работодателей указывают предлагаемую зарплату. Вероятно это связано с желанием работодателей оценить уровень кандидата при собеседовании и исходя из этого предлагать зарпалту. Можно предположить, что значительная часть вакансий представленных в базе относится к высокотехнологичным или требующим специальных навыков видам деятельности.
* Среднее значение зарплаты составляет: для нижней границы - 71065.0 рублей, для верхней границы - 110537.0 рублей. Учитывая, что более половины работодателей не указывают зарплату, а также достаточно большое количество регионов и возможных видов деятельности данные сведенья мало информативны. Уровень зарплаты стоит рассматривать для вакансий имеющих схожий набор ключевых навыков.
* Второе по популярности сочетание типа рабочего графика и типа трудоустройства: ['Удаленная работа' 'Полная занятость']. Такое сочетание встречается у 15,86% вакансий. Можно предположить, что в базе данных представлено достаточно много вакансий имеющих отношение к IT, хотя это могут быть и вакансии связанные с обзвонами или ответами на звонки.
* Наиболее популярное требование к опыту работы среди представленных вакансий от 1 года до 3 лет. Более половины работодателей (53,16%) ищут соискателей имеющих начальный опыт работы.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [25]:
query_5_1 = f'''(SELECT
                    e.name
                FROM public.vacancies AS v
                JOIN public.employers AS e 
                    ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                LIMIT 1)
                
                UNION ALL
                
                (SELECT
                    e.name
                FROM public.vacancies AS v
                JOIN public.employers AS e 
                    ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY COUNT(v.id) DESC
                OFFSET 4
                LIMIT 1)
            '''

In [39]:
pop_empl_1_5_place = pd.read_sql_query(query_5_1, connection)
print(f'На первом и пятом местах по количеству вакансий находятся '
      f'следующие работодатели: {pop_empl_1_5_place["name"].values[:]}')

На первом и пятом местах по количеству вакансий находятся следующие работодатели: ['Яндекс' 'Газпром нефть']


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [27]:
query_5_2 = f'''SELECT
                    a.name,
                    COUNT(DISTINCT e.id) AS cnt_employers,
                    COUNT(DISTINCT v.id) AS cnt_vacancies
                FROM public.areas AS a
                LEFT JOIN public.employers AS e
                    ON a.id = e.area
                LEFT JOIN public.vacancies AS v
                    ON a.id = v.area_id
                WHERE v.id IS NULL
                GROUP BY a.name
                ORDER BY cnt_employers DESC
                LIMIT 1
            '''

In [40]:
max_empl_null_vac = pd.read_sql_query(query_5_2, connection)
print(f'Наибольшее количество работодателей в регионах, где отсутствуют ' 
      f'вакансии, зарегестрированно в регионе: {max_empl_null_vac.iloc[0, 0]}')

Наибольшее количество работодателей в регионах, где отсутствуют вакансии, зарегестрированно в регионе: Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
query_5_3 = f'''SELECT 
                    e.name AS employer_name,
                    COUNT(DISTINCT v.area_id) AS count_area_vac
                FROM public.vacancies AS v
                JOIN public.employers AS e 
                    ON v.employer_id = e.id
                GROUP BY e.name
                ORDER BY count_area_vac DESC
            '''

In [30]:
cnt_areas_vac = pd.read_sql_query(query_5_3, connection)
# Выведем первые 5 строк для оценки данных 
display(cnt_areas_vac.head(5))

,employer_name,count_area_vac
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [31]:
query_5_4 = f'''SELECT
                    COUNT(e.id)
                FROM public.employers AS e
                LEFT JOIN public.employers_industries AS ei
                    ON e.id = ei.employer_id
                WHERE industry_id IS NULL
            '''

In [32]:
cnt_emp_no_ind = pd.read_sql_query(query_5_4, connection)
print(f'Количество работодателей с неуказанной сферой деятельности: {cnt_emp_no_ind.iloc[0,0]}')

Количество работодателей с неуказанной сферой деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [33]:
query_5_5 = f'''SELECT
                    e.name AS name_emp
                FROM public.employers AS e
                JOIN public.employers_industries AS ei 
                    ON e.id = ei.employer_id
                GROUP BY e.name
                HAVING COUNT(ei.industry_id) = 4
                ORDER BY name_emp
                OFFSET 2
                LIMIT 1
            '''

In [41]:
name_company = pd.read_sql_query(query_5_5, connection)
print(f'Название третьей компании в алфавитном списке, содержащем компании '
      f'с указаными четырьмя сферами деятельности: {name_company.iloc[0,0]}')

Название третьей компании в алфавитном списке, содержащем компании с указаными четырьмя сферами деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [42]:
query_5_6 = f'''SELECT
                    COUNT(ei.employer_id)
                FROM public.industries AS i
                JOIN public.employers_industries AS ei
                    ON i.id = ei.industry_id
                    AND i.name = 'Разработка программного обеспечения'
            '''

In [43]:
cnt_emp_soft_dev = pd.read_sql_query(query_5_6, connection)
print(f'Количество работодателей в качестве сферы деятельности которых '
      f'указана разработка программного обеспечения: {cnt_emp_soft_dev.iloc[0,0]}')

Количество работодателей в качестве сферы деятельности которых указана разработка программного обеспечения: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [44]:
# Получим список городов-милионников по предложенной ссылке.
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%' \
    + 'D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
# Воспользуемся библиотекой BeautifulSoup для извлечения таблицы по тегу и атрибуту класса.
page = BeautifulSoup(response.text, 'html.parser')
table = page.find_all('table', class_="standard sortable")

# Преобразуем результат в строку и разделим по символу двойных кавычек.
table = str(table)
table = table.split('"')

# Создадим пустой список и заполним его пройдясь в цикле по полученной строке и выбирая названия городов.
# Названиям городов предшествует тэг ' title=', однако заголовок таблицы имеет такой же тэг и заканчивается скобкой.
# Поэтому дополним условие отбора проверкой, что последний символ является буквой.
city_ls = []
for i in range(len(table)):
    if table[i] == ' title=' and table[i+1][-1].isalpha():
        city_ls.append(table[i+1])

# Преобразуем список в кортеж для использования в запросе и проверим его содержимое.
city_tpl = tuple(city_ls)
print(city_tpl)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [45]:
# Напишем запрос используя полученный списко городов-милионников
query_5_7 = f'''SELECT
                    a.name AS area,
                    COUNT(v.id) AS count_vacancies
                FROM public.vacancies AS v
                JOIN public.areas AS a 
                    ON v.area_id = a.id
                    AND a.name IN {city_tpl}
                JOIN public.employers AS e 
                    ON v.employer_id = e.id
                    AND e.name = 'Яндекс'
                GROUP BY a.name
                
                --Присоединяем строку с общим количеством вакансий
                UNION ALL 
                
                SELECT 
                    'total',
                    COUNT(v.id)
                FROM public.vacancies AS v
                JOIN public.areas AS a 
                    ON v.area_id = a.id
                    AND a.name IN {city_tpl}
                JOIN public.employers AS e 
                    ON v.employer_id = e.id
                    AND e.name = 'Яндекс'
                ORDER BY count_vacancies
            '''

In [46]:
yandex_vac = pd.read_sql_query(query_5_7, connection)
display(yandex_vac)

,area,count_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


### выводы по анализу работодателей
***
* В пятерке лидеров по количеству вакансий находятся IT-компания («Яндекс»), провайдер интернет услуг («Ростелеком»), два представителя банковского сектора («Тинькофф», «СБЕР») и добывающая компания («Газпром нефть»). Всем им в той или иной степени могут требоваться IT-специалисты.
* В базе присутствуют регионы, в которых нет вакансий, но в которых зарегестрированы работодатели. Среди таких регионов наибольшее количество работодателей зарегестрировано в регионе Россия. Вероятно часть работодателей имеют филиалы в различных городах, поэтому в графе регион регистрации указывают страну.  
* Лидером среди работодателей по географии предлагаемых вакансий является «Яндекс». Его вакансии предлагаются в 181 регионе. В базе представлено 192 работодателя которые предлагают вакансии в 10 и более регионах.
* Более трети работодателей (8419) не указывают сферу деятельности. Вероятно работодатели считают, что название компании говорит само за себя либо что сфера деятельности понятна из наименования вакансии и набора требуемых навыков.
* В базе представлено 1136 компаний указавших четыре сферы деятельности. Чаще всего указываются сферы деятельности относящиеся к различным стадиям жизненого цикла продукта (разработка, изготовление, продвижение/продажа, ремонт). Это говорит о том, что в базе присутствуют крупные компании способные обеспечить полный жизненый цикл своей продукции.  
* Более 15% (3553) работодателей явно указали в качестве сферы деятельности разработку программного обеспечения. Учитывая, что у трети работодателей совсем не указана сфера деятельности можно предположить что достаточно много предлагаемых в базе вакансий может быть связана с IT-сферой. Но лучше такую оценку проводить по требуемым ключевым навыкам и наименованию вакансии так как в других сферах деятельности тоже может быть потребность в IT-специалистах.    
* Компания «Яндекс» предлагает от 21 до 54 вакансий в городах-миллионниках России, всего 485 вакансий. В данной компании порядка 18000 сотрудников, количество вакансий в городах-миллионниках составляет 2,69% от этого числа. Можно предположить, что это количество вакансий связано с восполнением ушедших сотрудников. Чтобы сделать адекватный вывод о текучести кадров стоит подсчитать общее количество вакансий компании «Яндекс». Для этого воспользуемся второй частью нашего запроса, подсчитывающей общее число вакансий, и уберем условие нахождения вакансии в городах-миллионниках. В результате получим, что компания «Яндекс» предлагает 1993 вакансии, а это состовляет 10,74% от общего числа сотрудников. Для IT-сферы нормальным значением текучести кадров считается уровень в 8-10%. Интересно было бы рассмотреть показатель текучести персонала в динамическом разрезе. То есть, сравнить его с соответствующими показателями за более ранние периоды. Это позволит оценить насколько правильно выстроена кадровая политика в компании.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [47]:
query_6_1 = f'''SELECT
                    COUNT(id)
                FROM public.vacancies
                -- Выбираем названия содержащие 'data' или 'данн' независимо от регистра
                WHERE name ILIKE '%%data%%' OR name ILIKE '%%данн%%'
            '''

In [48]:
cnt_vac_data = pd.read_sql_query(query_6_1, connection)
print(f'Количество вакансий, имеющих отношение к данным: {cnt_vac_data.iloc[0][0]}')

Количество вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [49]:
# Запишем условие для вакансий на позицию дата-сайентиста в переменную для использования в запросах
ds_vac = '''(
             (
             name ILIKE '%%data scientist%%' 
             OR name ILIKE '%%data science%%'                    
             OR name ILIKE '%%исследователь данных%%'                    
             OR name ILIKE '%%машинн%%обучен%%'                    
             OR name ILIKE '%%machine learning%%'
             )
             /* 
               Для поиска вакансий содержащих 'ML' важно учитывать регистр
               поэтому используем LIKE вместо ILIKE 
             */  
             OR (
                 name LIKE '%%ML%%' AND name NOT LIKE '%%HTML%%'
                 )
            )
        '''
        
query_6_2 = f'''SELECT
                    COUNT(id)
                FROM public.vacancies
                WHERE {ds_vac}
                    AND (name ILIKE '%%junior%%' 
                        OR experience ILIKE '%%нет опыта%%' 
                        OR employment ILIKE '%%стажировка%%')
            '''

In [53]:
ds_jun_vac = pd.read_sql_query(query_6_2, connection)
print(f'В базе данных есть {ds_jun_vac.iloc[0][0]} вакансия, '
      f'подходящая для начинающего дата-сайентиста')

В базе данных есть 51 вакансия, подходящая для начинающего дата-сайентиста


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [51]:
query_6_3 = f'''SELECT 
                    COUNT(id)
                FROM public.vacancies
                WHERE {ds_vac}
                    AND (key_skills ILIKE '%%sql%%' 
                        OR key_skills ILIKE '%%postgres%%')
            '''

In [52]:
# результат запроса
sql_pg_vac = pd.read_sql_query(query_6_3, connection)
print(f'Количество вакансий для DS, в которых в качестве ключевого навыка указан '
      f'SQL или postgres: {sql_pg_vac.iloc[0][0]}')

Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [54]:
query_6_4 = f'''SELECT 
                    COUNT(id)
                FROM public.vacancies
                WHERE {ds_vac}
                    AND (key_skills ILIKE '%%python%%')
            '''

In [55]:
python_vac = pd.read_sql_query(query_6_4, connection)
print(f'Количество вакансий с требуемым ключевым навыком - '
      f'знание Python: {python_vac.loc[0][0]}')

Количество вакансий с требуемым ключевым навыком - знание Python: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [56]:
# Ключевые навыки разделены знаком табуляции функция CHR() возращает символ по коду из ASCII таблицы. 
# Код 9 сооветствует горизонтальной табуляции. Для определения количества ключевых навыков найдем 
# разность количества символов, посчитанное функцией LENGTH(), в исходной строке и строке с 
# исключенными знаками табуляции. Для замены табуляции на пустоту используем функцию REPLACE(). 
# Не забываем добавить 1 так как табуляцией разделяются как минимум два ключевых навыка.  
query_6_5 = f'''SELECT
                    ROUND(AVG(LENGTH(key_skills) \
                        - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1.0), 2)
                FROM public.vacancies
                WHERE {ds_vac} 
                    AND key_skills IS NOT NULL
            '''

In [57]:
avg_key_skills = pd.read_sql_query(query_6_5, connection)
print(f'Для DS вакансий в среднем указывается {avg_key_skills.iloc[0][0]} ключевых навыков')

Для DS вакансий в среднем указывается 6.41 ключевых навыков


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [58]:
query_6_6 = f'''SELECT
                    experience,
                    ROUND(AVG(
                        /*
                          Функция COALESCE попытается последовательно проверять, что переданные 
                          ей аргументы не являются значением null и вернет первый аргумент соответствующий 
                          этому условию
                        */
                        COALESCE((salary_from + salary_to)/2.0, salary_from, salary_to, 0)
                        ),0) AS avg_salary
                FROM public.vacancies
                WHERE {ds_vac} 
                    AND COALESCE((salary_from + salary_to)/2.0, salary_from, salary_to, 0) != 0
                GROUP BY experience
            '''

In [59]:
avg_salary_by_exp = pd.read_sql_query(query_6_6, connection)
print(f'Дата-сайнтист с опытом работы от 3 до 6 лет может рассчитывать '
      f'на среднюю зарплату: {round(avg_salary_by_exp.iloc[2][1])} рублей')

Дата-сайнтист с опытом работы от 3 до 6 лет может рассчитывать на среднюю зарплату: 243115 рублей


### выводы по предметному анализу
***
* Примерно 3,6 процента (1771) вакансий в базе имеют отношение к данным. Небольшой процент вакансий может говорить о малом размере базы. Для проверки рапространенности данного вида деятельности стоит провести оценку среди вакансий имеющих отношение к IT-сфере.  
* В базе данных есть 51 подходящая вакансия для начинающего дата-сайентиста. Это менее 3% от общего числа вакансий, имеющих отношение к данным. Достаточно мало вакансий возникают вопросы по путям получения опыта. Возможно участие/победы соискателя в хакатонах стоит расценивать как начальный опыт. 
* Для 201 вакансии на позицию DS в качестве ключевого навыка указан SQL или postgres. Относительно количества вакансий, отвечающих выбранному критерию отнесения вакансии к DS, навыки работы с базами данных требуются в 41,9% случаев. Стоит учесть, что не для всех вакансий указываются ключевые навыки, некоторые работодатели подразумевают наличие навыков в наименовании самой вакансии.
* Для 351 вакансии на позицию DS в качестве ключевого навыка указано знание Python. Относительно количества вакансий, отвечающих выбранному критерию отнесения вакансии к DS, знание Python требуется в 73,1% случаев. Действительно Python является наиболее распространненым языком программирования в DS, однако, встречаются вакансии требующие знание С++ или Java. Также стоит учитывать, что иногда требование к ключевым навыкам подразумевается в наименовании вакансии.
* Для DS вакансий в среднем указывается 6.41 ключевых навыков. Работодатели наравне с общими навыками ожидают и знания из конкретных областей. Например, кроме знания Python указывается владение библиотками Pandas, Numpy и другие. Возможно для рекомендательной системы стоит заострить внимание на основных навыках, а знание в конкретных областях не учитывать при подборе вакансий. То есть, например, соискатель может явно не указать владение библиотеками Python считая это само собой разумеющимся.
* Дата-сайнтист с опытом работы от 3 до 6 лет может рассчитывать на среднюю зарплату: 243115 рублей. Не смотря на то, что не все работодатели указывают уровень зарплат данный показатель может использоваться для оценки уровня зарплаты в привязке к опыту работы. Кроме опыта работы можно рассмотреть зависимость от региона в котором предложена вакансия. Интересно будет посмотреть зависимость от типа рабочего графика.

# Дополнительные исследования
***

1. Посмотрим на распределение работодателей по сферам деятельности.

In [70]:
# Получим распределение работодателей по сферам деятельности.
query_7_1 = f'''SELECT
                    i.name AS industries,
                    COUNT(e.id) AS count_emp,
                    -- Для подсчета доли от общего числа воспользуемся подзапросом
                    ROUND(100.0*COUNT(e.id)/(SELECT COUNT(id) FROM public.employers), 2) AS percent
                FROM public.employers AS e
                JOIN public.employers_industries AS ei 
                    ON e.id = ei.employer_id
                JOIN public.industries AS i 
                    ON ei.industry_id = i.id
                GROUP BY i.name
                ORDER BY count_emp
            '''
ind_emp = pd.read_sql_query(query_7_1, connection)
# Посмотрим на результат.
display(ind_emp)

# Выведем пример професий не относящихся к ИТ.
query_7_2 = f'''SELECT 
                    COUNT(*)
                FROM public.vacancies
                WHERE name ILIKE '%%столяр%%'
            '''
not_it_ind = pd.read_sql_query(query_7_2, connection)
display(f'Количество вакансий имеющих отношение к столярному делу: {not_it_ind.iloc[0][0]}')

# Строим столбчатую диаграмму.
fig = px.treemap(
    ind_emp,
    path=['industries'],
    values='percent',   
    height=500,
    width=1000
    )
# Настраиваем отображение всплывающего текста и названия графика.
fig.update_traces(hovertemplate='%{label}<br>Процент от общего числа работодателей: %{value}%<extra></extra>')
fig.update_layout(title=dict(
                             text='Процентное распределение работодателей по сферам деятельности',
                             x=0.5,
                             y=0.95,
                             font=dict(size=24)
                             ),
                  margin=dict(t=60, l=10, r=10, b=10)
                  )
fig.show()

,industries,count_emp,percent
0,"Ассоциация в сфере культуры, искусства",1,0.00
1,Автошкола,3,0.01
2,"Ботанический сад, зоопарк, заповедник",3,0.01
3,"Лифтовое хозяйство (монтаж, сервис, ремонт)",4,0.02
4,Ритуальные услуги,4,0.02
...,...,...,...
289,Консалтинговые услуги,662,2.82
290,"Маркетинговые, рекламные, BTL, дизайнерские, E...",798,3.40
291,"Интернет-компания (поисковики, платежные систе...",1675,7.13
292,"Системная интеграция, автоматизации технологи...",2993,12.74


'Количество вакансий имеющих отношение к столярному делу: 11'

***
Распределение работодателей по сферам деятельности показывает, что в тройку лидеров входят работодатели напрямую связанные с IT-сферой ("Разработка программного обеспечения", "Системная интеграция, автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг", "Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)"). Можно сделать вывод, что преимущественно база содержит сведенья о вакансиях связанных с IT. Однако, в базе присутствуют профессии не относящиеся к IT. Например, 11 вакансий имеют отношение к столярному делу. Возможно стоит провести доплонительную очистку данных раз речь идет о рекомендательной системе для IT професий.

2. Посмотрим на количество вакансий у которых в качестве региона указана страна.

In [73]:
# Получим список стран мира с сайта wikipedia.
url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%81%\
D1%83%D0%B4%D0%B0%D1%80%D1%81%D1%82%D0%B2'
country_list = tuple(pd.read_html(url)[0]['Страна'])

# Напишем запрос выводящий вакансии, предложенные в странах для различных типов рабочего графика,
# а также общее число вакансий, предлагаемых в странах.
query_7_3 = f'''(SELECT
                    a.name AS location,
                    COUNT(v.id) AS cnt_vac,
                    v.schedule
                FROM public.vacancies AS v
                JOIN public.areas AS a
                    ON v.area_id = a.id
                    AND a.name IN {country_list}
                GROUP BY a.name, v.schedule
                ORDER BY schedule ASC, location ASC)  
                                
                UNION ALL
                
                SELECT
                    'Total',
                    COUNT(v.id),
                    'All_schedule'
                    FROM public.vacancies AS v
                    JOIN public.areas AS a
                        ON v.area_id = a.id
                        AND a.name IN {country_list}
            '''
out_vac = pd.read_sql_query(query_7_3, connection)

# Выведем общее количество таких вакансий
display(f'Общее количество вакансий у которых в качестве региона указана страна: {out_vac.iloc[0][1]}')

# Построим столбчатую диаграмму.
fig = px.bar(
    out_vac.iloc[1:],
    x='location',
    y='cnt_vac',
    height=500,
    width=1000,
    color='schedule',
    barmode='group',
    labels={'location':'страна', 'schedule':'тип рабочего графика', 'cnt_vac':'количество вакансий'}
    )

fig.update_layout(title=dict(
                             text='Количество вакансий у которых в качестве региона указанна страна',
                             x=0.5,
                             font=dict(size=24)
                             ),
                  margin=dict(t=90),
                  legend=dict(title_text='Тип рабочего графика:'),
                  yaxis1=dict(gridwidth=1, gridcolor='gray'),
                  
                  )
fig.show()

'Общее количество вакансий у которых в качестве региона указана страна: 632'

***
 В базе присутствуют вакансии регионом которых указана страна (632 вакансии), причем тип рабочего графика "удаленная работа" указан только у 221 вакансии. В большинстве случаев предлагается тип рабочего графика - "Полный день". В принципе такие вакансии могут попасть в рекомендуемую выборку, например, если соискатель указывал готовность к переездам.

3. Посмотрим на вакансии Data Scientist в разрезе типа рабочего графика и опыта работы.

In [74]:
# Посмотрим на соотношение вакансий на позицию DS в зависимости от требований к опыту с типом 
# рабочего графика "Удаленная работа" и "Работа в офисе". За "Работу в офисе" примем все типы 
# рабочих графиков кроме "Удаленная работа".

query_7_4 = f'''SELECT
                    'Удаленная работа' AS "Рабочий график",
                    experience AS "Опыт",
                    count(id) AS "Количество вакансий"
                FROM public.vacancies
                WHERE 
                    schedule ILIKE '%%удал%%'
                    AND {ds_vac}
                GROUP BY experience
                
                UNION ALL
                
                SELECT
                    'Работа в офисе' AS "Рабочий график",
                    experience AS "Опыт",
                    count(id) AS "Количество вакансий"
                FROM public.vacancies
                WHERE 
                    schedule NOT ILIKE '%%удал%%'
                    AND {ds_vac}
                GROUP BY experience
                ORDER BY "Опыт", "Рабочий график"
            '''

df_shedule = pd.read_sql_query(query_7_4, connection)
display(df_shedule)

# Строим столбчатую диаграмму
fig = px.bar(
    data_frame=df_shedule,
    height=500,
    width=940,
    x='Опыт',
    y='Количество вакансий',
    color='Рабочий график'    
    )

# Настраиваем отображение
fig.update_layout(
                  title=dict(
                             text='Диаграмма зависимости типа рабочего графика от'\
                                 +'опыта работы для вакансий на позицию Data Scientist',
                             x=0.5,
                             font=dict(size=18)
                             ),   
                  xaxis_title='Опыт',
                  yaxis_title='Количество вакансий',
                  legend=dict(title='Тип рабочего графика')
                  )

fig.show()


,Рабочий график,Опыт,Количество вакансий
0,Работа в офисе,Более 6 лет,7
1,Удаленная работа,Более 6 лет,27
2,Работа в офисе,Нет опыта,25
3,Удаленная работа,Нет опыта,3
4,Работа в офисе,От 1 года до 3 лет,182
5,Удаленная работа,От 1 года до 3 лет,41
6,Работа в офисе,От 3 до 6 лет,151
7,Удаленная работа,От 3 до 6 лет,44


 ***
 Из диаграммы видно, что с ростом опыта растет и предлагаемое количество вакансий с типом рабочего графика "Удаленная работа". Видимо работодатели с большим доверием относяться к соискателям с опытом работы, считая их более ответственными. Стоит рассмотреть распределение вакансий по городам.

4. Исследуем распределение вакансий на позицию Data Scientist по городам России.

In [77]:
# Чтобы посмотреть распределение вакансий по городам России получим список городов с сайта wikipedia.  
url = 'https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_\
%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8_%D1%81_\
%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC_%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5_\
100_%D1%82%D1%8B%D1%81%D1%8F%D1%87_%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9'
    
cities=pd.read_html(url)[0].iloc[:, 2:3]

# Преобразуем датафрейм в список.
city_list = list(cities['Город']['Город'])

# В полученном списке для некоторых городов указаны регионы в квадратных скобках. 
# Избавимся от них и получим кортеж с названиями городов России.
for ind, val in enumerate(city_list):
    if '[' in val:
        val = val.split('[')
        city_list[ind] = val[0]
city_list = tuple(city_list)

# Напишем запрос для получения количества вакансий на позицию Data Scientist в городах России.
query_7_5 = f'''SELECT
                    a.name AS city,
                    COUNT(v.id) AS cnt_vac
                FROM public.vacancies AS v
                JOIN public.areas AS a
                    ON v.area_id = a.id
                    AND a.name NOT IN {country_list}
                    AND a.name IN {city_list}
                /* 
                Так как используется соединение таблиц содержажих одинаковые наименования полей требуется
                указывать алиесы, поэтом условие отнесения вакансии к DS нужно прописать заново.
                */
                WHERE (v.name ILIKE '%%data scientist%%'                    
                    OR v.name ILIKE '%%data science%%'                    
                    OR v.name ILIKE '%%исследователь данных%%'                    
                    OR v.name ILIKE '%%машинн%%обучен%%'                    
                    OR v.name ILIKE '%%machine learning%%')
                    OR (v.name LIKE '%%ML%%' AND v.name NOT LIKE '%%HTML%%')                    
                GROUP BY a.name
                ORDER BY cnt_vac DESC
            '''

ds_vc = pd.read_sql_query(query_7_5, connection)

# Чтобы отобразить распределение вакансий на крате нам потребуется дополнить полученную 
# таблицу координатами городов. Для чего вытащим их по ссылке.
url = 'https://on55.ru/statya/koordinaty-lat-i-lng-vsex-gorodov-rossii'
coordinats = pd.read_html(url)[0]

# Выберем интересующие нас данные и переименуем столбцы.
city_coord = pd.concat([coordinats.iloc[:, 0], coordinats.iloc[:, 3:5]], axis=1)
city_coord.columns = ['city', 'geo_lat', 'geo_lon']
city_coord = city_coord.drop([0]) # Убираем первую строку со старыми названиями столбцов

# В полученной таблице составные названия городов написаны слитно. Например, 'ПавловскийПосад'.
# Разделяем названия городов по заглавной букве.
city_coord['city'] = city_coord['city'].apply(lambda x: ' '.join(re.sub(r'([А-Я])', r' \1', x).split()))

# Убирем получившиеся пробелы из названий городов с дефисом с помощью функции.
def city_fix(name):
    if '- ' in name:
        name = '-'.join(name.split('- '))
        return name
    return name

city_coord['city'] = city_coord['city'].apply(city_fix)

# Изменяем тип данных для координат со строкового на числовой.
city_coord = city_coord.astype({'geo_lat':np.float32})
city_coord = city_coord.astype({'geo_lon':np.float32})

# В таблице с координатами обнаружилась ошибка в координатах Нижнего Новгорода. 
# Исправим это заменив координаты на верные.
city_coord.loc[city_coord['city'] == 'Нижний Новгород', 'geo_lat'] = 56.3287
city_coord.loc[city_coord['city'] == 'Нижний Новгород', 'geo_lon'] = 44.002

# Объединяем наши таблицы в одну по столбцу 'city'.
city_tab = ds_vc.merge(city_coord, on = 'city', how='left')

# Строим карту с распределением количества вакансий по городам.
fig = go.Figure(go.Scattermapbox(
                                lat=city_tab['geo_lat'],
                                lon=city_tab['geo_lon'],
                                mode='markers+text',
                                marker=dict(
                                            colorbar=dict(
                                                          title=dict(
                                                                     text='Количество вакансий',
                                                                     side='right'
                                                                     ),
                                                          xanchor='left',
                                                          xpad=20
                                                          ),
                                            color=city_tab['cnt_vac'],
                                            cmin=-100,
                                            cmax=250,                                    
                                            colorscale = 'spectral',                                           
                                            size=city_tab['cnt_vac'],
                                            sizemode='area',
                                            sizeref=0.7                               
                                            ),
                                textposition='top right',
                                textfont=dict(size=16, color='black'),
                                text=[
                                      city_tab['city'][i] + '<br>' + 'Количество вакансий: ' +
                                      str(city_tab['cnt_vac'][i]) for i in range(city_tab.shape[0])
                                      ]
                                )
                )
               
# Настроим центркарты как среднее значение между максимальными и минимальными значениями 
# координат городов из нашей таблицы. 
map_center = go.layout.mapbox.Center(
                                     lat=(city_tab['geo_lat'].max()+city_tab['geo_lat'].min())/2, 
                                     lon=(city_tab['geo_lon'].max()+city_tab['geo_lon'].min())/2
                                     )
fig.update_layout(
                  mapbox_style="carto-darkmatter",
                  mapbox=dict(center=map_center, zoom=2),
                  height=500, 
                  width=1500,
                  margin=dict(r=10, b=20, t=60, l=10),
                  title=dict(
                             text='Количество вакансий на позицию DataScientist в городах России',
                             x=0.5,
                             y=0.97,
                             font=dict(size=28)
                             )                  
                  )

fig.show()


***
 Распределение вакансий по городам показывает, что основное количество ваканси на позиция Data Scientist сосредоточено в Москве и Санкт-Петербурге 217 и 64 вакансии соответственно. Далее идут Новосибирск, Нижний Новгород и Казань с количеством вакансий от 16 до 23. В целом, с учетом того что без опыта работы вакансию с типом рабочего графика "Удаленная работа" найти очень трудно, начинающему дата сайентисту стоит подумать о переезде в Москву или Санкт-Петербург.

5. Посмотрим на средний уровень зарплат для вакансий Data Scientist в заисимости от города и опыта работы.

In [76]:
# Проверим средний уровень зарплат предлагаемых для вакансий Data Scientist в городах России в 
# зависимости от опыта работы.
query_7_6 = f'''SELECT
                    a.name AS city,
                    experience,
                    ROUND(AVG(
                        COALESCE((salary_from + salary_to)/2.0, salary_from, salary_to, 0)
                        ),0) AS avg_salary
                FROM public.vacancies AS v
                LEFT JOIN public.areas AS a
                    ON v.area_id = a.id
                WHERE (
                    (v.name ILIKE '%%data scientist%%'
                    OR v.name ILIKE '%%data science%%'
                    OR v.name ILIKE '%%исследователь данных%%'
                    OR v.name ILIKE '%%машинн%%обучен%%'
                    OR v.name ILIKE '%%machine learning%%')
                    OR (v.name LIKE '%%ML%%' 
                    AND v.name NOT LIKE '%%HTML%%') 
                    ) 
                    AND COALESCE((salary_from + salary_to)/2.0, salary_from, salary_to, 0) != 0
                GROUP BY experience, a.name
            '''
x = pd.read_sql_query(query_7_6, connection)

# Объединим результат запроса с предыдущей таблицей, чтобы оставить данные только для городов России. 
df = city_tab.merge(x, on = 'city', how='left')

# Построим столбчатую диаграмму.
fig = px.bar(
             df,
             x='city',
             y='avg_salary',
             height=500,
             width=1000,
             color='experience',
             barmode='group',
             labels={'city':'Город', 'experience':'Опыт', 'avg_salary':'Средняя зарплата в руб.'}
             )

# Повышаем информативность
fig.update_layout(title=dict(
                             text='Предлагаемый уровень зарплаты для вакансий Data Scientist в городах России',
                             x=0.5,
                             font=dict(size=24)
                             ),
                  margin=dict(t=90),
                  legend=dict(title_text='Опыт:'),
                  yaxis1=dict(gridwidth=1, gridcolor='gray')                  
                  )
fig.show()


 ***
 Неожиданным лидером по среднему уровню предлагаемой зарплаты на вакансии Data Scientist для всех категорий опыта оказался Санкт-Петербург. Если рассматривать категорию "Нет опыта" то после Санкт-Петербурга идет Новосибирск (80 тысяч рублей), Москва (72,5 тысячи рублей), Владивосток (70 тысяч рублей) и Екатеринбург (37 тысяч рублей). Можно предположить, что наиболее перспективными городами с точки зрения начала карьеры Data Scientis являются Санкт-Петербург,  Новосибирск, Москва и Владивосток. С получением опыта появляется больше вакансий с возможностью удаленной работы и город проживания уже не играет большой роли. Странно низкая зарплата в Екатеринбурге, при достаточно высокой в других категориях, может говорить о выбросах в данных. Для продолжения работы можно предложить проверить данные на выбросы и провести очистку при необходимости. 

# Общий вывод по проекту
***

Подведем итоги: 
* в нашем распоряжении на первый взгляд достаточно большая подборка данных на 49197 вакансий. Однако согласно википедии среднее дневное количество вакансий на  HeadHunter (hh.ru) превышает 933 тыс. Общее количество вакансий на позицию  Data Scientist  не вилико - всего 480 вакансий, что также приводит к мысли о малом размере подборки данных. Кроме того в данных присутствуют и вакансии не имеющие отношения к IT сфере. Встречаются вакансии регионом которых указана страна и не все из них предлагают удаленную работу. Также мы увидели ннизкую среднюю зарплату в Екатеринбурге для вакансий Data Scientist без опыта работы, при достаточно высокой зарплате для других категорий опыта. Все это говорит о наличии выбросов, для продолжения работы можно предложить провести очистку данных. Так же можно рекомендовать рассмотреть категоризацию ключевых параметров. Например, сгруппировать знание специализированных библиотек со знанием языка. Ведь соискатель может указать только знание Python подразумевая знание библиотек. Думаю, что для рекомендательной системы такое допущение приемлемо;       
                                 
* по результатам анализа предложенных в подборке вакансий Data Scientist можно сказать, что для начала карьеры стоит рассмотреть переезд в Санкт-Петербург, Новосибирск или Москву. Возможно стоит рассмотреть Нижний Новгород или Казань, но данные по зарплате в этих городах отсутствуют. В остальных городах очень мало вакансий для соискателей без опыта или низкий уровень зарплаты. Стоит отметить, что с ростом опыта растет и количество предлагаемых вакансий с типом рабочего графика "Удаленная работа" и город проживания уже не играет решающей роли.